## FEniCSx implementation

### The complete program

A FEniCSx program for solving our test problem for the Poisson equation
in 2D with the given choices of $\Omega$, $u_D$, and $f$ may look as
follows:

In [ ]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, plot, io
from dolfinx.fem.petsc import LinearProblem
from ufl import SpatialCoordinate, TrialFunction, TestFunction, inner, grad, dx

# Create mesh and define function space
msh = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    nx=4,
    ny=4
)

V = fem.functionspace(
    mesh=msh,
    element=("P", 1)
)

# Define boundary condition
def on_boundary(x):
    return np.isclose(x[0], 0) | np.isclose(x[0], 1) | np.isclose(x[1], 0) | np.isclose(x[1], 1)

boundary_dofs = fem.locate_dofs_geometrical(V=V, marker=on_boundary)

def manufactured_solution(x):
    return 1 + x[0]**2 + 2 * x[1]**2

uD = fem.Function(V)
uD.interpolate(manufactured_solution)

bc = fem.dirichletbc(value=uD, dofs=boundary_dofs)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = fem.Constant(msh, -6.)
a = inner(grad(u), grad(v)) * dx
L = f * v * dx

# Compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()
uh.name = "Solution u"

# Compute error
x = SpatialCoordinate(msh)
ue = manufactured_solution(x)
L2form = fem.form((uh - ue)**2 * dx)
L2error = np.sqrt(fem.assemble_scalar(L2form))
print("L2-error:", L2error)
H1form = fem.form((uh - ue)**2 * dx + inner(grad(uh - ue), grad(uh - ue)) * dx)
H1error = np.sqrt(fem.assemble_scalar(H1form))
print("H1-error:", H1error)

# Export the solution in VTX format
with io.VTXWriter(msh.comm, "results/poisson.bp", [uh]) as vtx:
    vtx.write(0.0)

### Convergence with $P_1$-elements

Use the code above with the manufactured solution $u_e(x,y) = 1 + x^2 + 2y^2$ to complete the following table for the approximation with $P_1$-elements:

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $4$ | ${1 \over 4} \sqrt{2}$ | 0.0329 | 0.3244 |
| $8$ | ${1 \over 8} \sqrt{2}$ | 0.0082 | 0.1616 |
| $16$ | ${1 \over 16} \sqrt{2}$ | 0.0021 | 0.0807 |
| $32$ | ${1 \over 32} \sqrt{2}$ | 0.0005 | 0.0403 |
| $64$ | ${1 \over 64} \sqrt{2}$ | 0.0001 | 0.0201 |
| $128$ | ${1 \over 128} \sqrt{2}$ | 0.00003 | 0.0100 |

Q: What are the convergence rates that these results suggest for the $L^2$-error and the $H^1$-error, respectively?

A: 

Now switch from $P_1$-elements to $P_2$-elements and check that the numerical solution is exact (up to rounding error).

### Convergence with higher-order elements

Create a new test problem by choosing a quintic polynomial as manufactured solution:

$$u_e(x,y) = $$

Q: What source term $f$ and what boundary values $u_D$ make this the exact solution of the Poisson-Dirichlet problem

\begin{align*}
-\Delta u (x,y) &= f(x,y) && (x,y) \in \Omega\\
u (x,y) &= u_D(x,y) && (x,y) \in \partial\Omega
\end{align*}

A: 

\begin{align*}
f(x,y) &= \\
u_D(x,y) &= 
\end{align*}

Implement this new manufactured solution in the code above. Then measure the $L^2$-errors and $H^1$-errors when using $P_1$, $P_2$ or $P_3$ elements:

#### $P_1$-elements

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $8$ |  |  |
| $16$ |  |  |
| $32$ |  |  |
| $64$ |  |  |

Estimated convergence rates:
- ... convergence for the $L^2$ errors
- ... convergence for the $H^1$ errors

#### $P_2$-elements

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $8$ |  |  |
| $16$ |  |  |
| $32$ |  |  |
| $64$ |  |  |

Estimated convergence rates:
- ... convergence for the $L^2$ errors
- ... convergence for the $H^1$ errors

#### $P_3$-elements

| $n_x, n_y$ | $h$ | $L^2$-error | $H^1$-error |
| --- | --- | --- | --- |
| $8$ |  |  |
| $16$ |  |  |
| $32$ |  |  |
| $64$ |  |  |

Estimated convergence rates:
- ... convergence for the $L^2$ errors
- ... convergence for the $H^1$ errors